In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [2]:
train_data.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,...,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,...,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,...,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,...,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,...,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0


In [3]:
train_data['Class'].value_counts()

0    218660
1       469
Name: Class, dtype: int64

In [4]:
X_train = train_data.drop(columns=['Class'])
y_train = train_data['Class']
X_test = test_data

In [5]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

X_train['log_amount'] = np.log1p(X_train['Amount'])
X_test['log_amount'] = np.log1p(X_test['Amount'])

model = XGBClassifier(objective='binary:logistic', eval_metric='logloss')

rfe = RFE(estimator=model, n_features_to_select=10)  # Set the number of features to select
rfe.fit(X_train, y_train)

selected_features = X_train.columns[rfe.support_]
print("Selected Features: ", selected_features)

Selected Features:  Index(['V3', 'V7', 'V8', 'V10', 'V12', 'V14', 'V17', 'V19', 'V26', 'Amount'], dtype='object')


In [6]:
X_train = X_train.drop(columns=['V1', 'V2', 'V4', 'V5', 'V6', 'V9', 'V11', 'V13', 'V15', 'V16', 'V18', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V27', 'V28'])
X_test = X_test.drop(columns=['V1', 'V2', 'V4', 'V5', 'V6', 'V9', 'V11', 'V13', 'V15', 'V16', 'V18', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V27', 'V28'])

In [7]:
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss')

model.fit(X_train, y_train)

predictions = model.predict(X_test)
X_test['PredictedClass'] = predictions

X_test

,id,Time,V3,V7,V8,V10,V12,V14,V17,V19,V26,Amount,log_amount,PredictedClass
0,219129,120580.0,-1.305514,-0.924262,-0.023030,0.220610,-0.723363,-2.243711,1.213534,0.579121,-0.084889,29.95,3.432373,0
1,219130,120580.0,-0.547387,-1.632859,1.073529,0.483337,-0.346514,-0.564673,0.160901,-0.066264,0.068708,163.50,5.102911,0
2,219131,120580.0,-0.238965,-1.682064,0.105613,1.626352,-0.041610,-0.527684,0.058472,0.010671,-0.297813,16.00,2.833213,0
3,219132,120580.0,-1.938259,0.252889,-0.399789,-1.225592,1.647638,-0.258468,0.099479,1.207085,-0.627529,120.98,4.803857,0
4,219133,120580.0,0.363601,0.039709,-0.048895,-0.341879,0.637604,0.331702,-0.853439,-0.267098,-0.025706,1.98,1.091923,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146082,365211,172789.0,0.735580,0.161226,0.069945,-0.658935,-0.655069,-0.143536,-0.949933,0.079083,0.161938,94.81,4.562367,0
146083,365212,172789.0,0.340012,0.726272,-0.070608,-0.776078,0.506635,-0.039782,-0.746027,-0.191256,-0.561426,24.99,3.257712,0
146084,365213,172789.0,0.551671,0.567545,-0.059899,1.623577,1.021134,-0.322923,-0.764603,0.291846,-0.213933,29.99,3.433665,0
146085,365214,172790.0,0.068496,0.981608,-0.209713,-0.283025,-0.559516,-0.887569,0.114537,0.664771,-0.204957,314.00,5.752573,0


In [8]:
X_test['PredictedClass'].value_counts()

0    146063
1        24
Name: PredictedClass, dtype: int64

In [10]:
results = X_test[['id', 'PredictedClass']]

In [11]:
results.head()

,id,PredictedClass
0,219129,0
1,219130,0
2,219131,0
3,219132,0
4,219133,0


In [14]:
results.to_csv('results.csv', index=False)